In [1]:
import sys
from functions.analysis import *
from functions.models import *
configuration = {
    'type': 'Dense',
    'activation': 'relu',
    'hidden_layers': 1,
    'input_shape': 1,
    'hidden_size': 30,
    'use_bias': True,
    'last_size': 1,

    # Training Parameters
    'batch_size': 500,
    'epochs': 100,
    'learn_rate': 0.005,
    'loss': 'mse',
    'metrics': tf.keras.metrics.MSE,
    'last_activation': None,

    'fname_savings': 'Test.pk'
}
if configuration['hidden_layers'] != 1:
    raise ValueError('Hidden layers at most 1!')

(x_train, y_train), (x_test, y_test) = dataset_loader(dataset_name='function',
                                                      function=lambda x: x*(1-x**2))

sizes = [configuration['input_shape'],
         configuration['hidden_size'],
         configuration['last_size']]

cc, B1, B2, imB1t, imB2 = bases_retrive(sizes,
                                        return_base=True,
                                        decouple=True)


C:\Users\loren\PycharmProjects\HodgeDNN\functions\analysis.py:206: UserWarning: Decoupling with structure assumed 2 lay feedforward
  warn('Decoupling with structure assumed 2 lay feedforward')


In [2]:
model = cohomology_train(sizes=sizes,
                     imB1t=imB1t,
                     imB2=imB2,
                     n_vet_B1t=100,
                     n_vet_B2=100,
                     config=configuration)

model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=configuration['learn_rate']),
          loss=configuration['loss'],
          metrics=configuration['metrics'],
          run_eagerly=False)
print('Training...')
model.fit(x_train, y_train,
      epochs=configuration['epochs'],
      #callbacks=retrive_coefficients(),
      verbose=0)

cc = feedforward_CC(sizes)
loss_before, binacc_before = model.evaluate(x_test, y_test)
old_coef = np.pad(model.coef_B2.numpy().reshape([-1]), (0, model.imB2.shape[1]))

Training...
32/32 [==============================] - 0s 2ms/step - loss: 2.4381e-05 - mean_squared_error: 2.4381e-05


In [3]:
from ipywidgets import FloatSlider, interact
from IPython.display import display
from functions.plotting import *

def plot_interactive(line_range):
    interval = [line_range-0.1, line_range]
    new_x, new_y = function_modifier(x_train.copy(),
                            y_train.copy(),
                            lambda x: -x ** 0.6,
                            interval)
    model.fit(new_x, new_y,
                epochs=20,
                verbose=0)
    fig, (func, axes) = plt.subplots(1, 2, figsize=(9, 3), dpi=200)
    drawer = NN_drawer(cc)

    func.plot(np.sort(new_x, axis=0), new_y[np.argsort(new_x, axis=0)],
              '-r',
              markersize=1,
              alpha=0.8,
              label='Ground',
              zorder=3)
    
    func.plot(new_x, model(new_x).numpy(),
              'bo',
              markersize=2,
              alpha=0.8,
              label='HodgeNN')

    func.axvline(x=interval[0], ymin=-1, ymax=1)
    func.axvline(x=interval[1], ymin=-1, ymax=1)
    axes1, _ = drawer.plot_diff(coef0=old_coef, model1=model)
    plt.show()

x_widget = FloatSlider(min=-0.9, max=1, step=0.1)

inter = interact(plot_interactive, line_range=x_widget)
inter

interactive(children=(FloatSlider(value=0.0, description='line_range', max=1.0, min=-0.9), Output()), _dom_cla…

<function __main__.plot_interactive(line_range)>

In [5]:
abs(model.coef_B2.numpy()).mean()

0.5402085